# 📜 Historical Handwritten Language Translator (HTR + NLP)

### ✅ What this notebook demonstrates
This notebook builds a complete pipeline for historical manuscript digitization:

**Image Upload → Handwriting Transcription (Qwen2.5-VL) → Text Refinement + Modern Translation (Groq LLaMA) → Streamlit Web App → Public Link via Cloudflare Tunnel**

---

### 🧩 Key components
- **Qwen2.5-VL (Vision-Language Model)**: reads handwritten manuscripts from images (Chinese / Old English)
- **Groq (LLM API inference)**: refines transcription + performs modernization/translation
- **Streamlit**: lightweight UI for demo
- **Cloudflare Tunnel**: exposes the app publicly without deployment

> ⚠️ Note: This project uses **pretrained models** (no training dataset required). It runs in **inference-only** mode.


##  Installing Dependencies & Setting Up Environment

This section installs all required libraries:
- Streamlit for UI
- PyTorch + Transformers for model loading
- BitsAndBytes for 4-bit quantization
- Groq SDK for LLM API calls
- Qwen VL utilities for multimodal processing


# **1. Installing Dependencies & Setup Environment**

In [1]:
import os

# Installing core libraries
!pip install -q streamlit pyngrok torch torchvision transformers accelerate
!pip install -q opencv-python-headless pillow
!pip install -q python-dotenv
!pip install -U bitsandbytes>=0.46.1
!pip install groq
# Install Qwen-VL specific utilities (Critical for Arabic HTR)
!pip install -q qwen-vl-utils git+https://github.com/huggingface/transformers

print("✅ Environment Installed. Ready for Model Setup.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 14.2 MB/s eta 0:00:00
✅ Environment Installed. Ready for Model Setup.


##  Creating the Streamlit Web Application (`app.py`)

This cell writes the full Streamlit app into a Python file.
The app contains:

### Phase 1: HTR (Handwritten Text Recognition)
- Uses **Qwen2.5-VL** to transcribe handwriting from the uploaded image.

### Phase 2: NLP Refinement + Translation
- Uses **Groq API (LLaMA 3.3 70B)** to:
  - Correct transcription errors
  - Expand abbreviations (Old English)
  - Output modern English translation


# **2. Creating the Streamlit App**

In [2]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
import os
from groq import Groq
# --- 0. MEMORY OPTIMIZATION ---
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- PAGE CONFIGURATION ---
st.set_page_config(page_title="Historical Handwritten Language Translator (HTR & NLP)", layout="wide", page_icon="📜")



# --- DEVICE SETUP ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cpu":
    st.sidebar.warning("⚠️ Running on CPU. Switch to T4 GPU runtime for speed.")

# --- 1. MODEL LOADING (4-BIT QUANTIZED)---
@st.cache_resource
def load_qwen_model():
    print("Loading Qwen2.5-VL (with 4-bit Quantization)...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2.5-VL-3B-Instruct",
        quantization_config=bnb_config,
        device_map="auto",
    )
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
    return processor, model

# --- 2. PROCESSING FUNCTIONS ---

def resize_image_if_needed(image, max_size=1280):
    width, height = image.size
    if width > max_size or height > max_size:
        image.thumbnail((max_size, max_size))
    return image

def extract_text_qwen(image, processor, model, mode):
    image = resize_image_if_needed(image)

    # --- PROMPTS FOR CHINESE ---
    if mode == "Chinese":
        # Qwen is S-Tier at Chinese; we ask for precise character transcription
        prompt_text = "Transcribe the handwritten Chinese text in this image exactly. Identify specific characters (Traditional or Simplified) as written. Do not add commentary."
    else:
        prompt_text = "Transcribe this handwritten English manuscript exactly. Maintain line breaks and original spelling."

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt_text},
            ],
        }
    ]

    text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text_input],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=1024) # Increased token limit for dense Chinese text
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return output_text


def process_with_llm(text, mode):
    api_key ='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'
    client = Groq(api_key=api_key)

    # Llama 3.3 70B is excellent for reasoning/translation and is free on Groq
    model_id = "llama-3.3-70b-versatile"

    if mode == "Chinese":
        system_prompt = "You are an expert translator of Chinese Calligraphy."
        user_prompt = f"""
        1. Correct any OCR errors in this text: "{text}"
        2. Translate it into Modern English.

        Output format:
        **Corrected Chinese:**
        [text]
        **Modern Translation:**
        [text]
        """
    else:
        system_prompt = "You are an expert paleographer specializing in 18th-century manuscripts."
        user_prompt = f"""
        1. Correct any OCR errors in this text (it is 18th-century English): "{text}"
        2. Expand abbreviations (e.g., "&c" -> "etc.").
        3. Translate/Modernize it into clear Modern UK English.

        Output format:
        **Corrected Transcription:**
        [text]
        **Modern Translation:**
        [text]
        """

    try:
        completion = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1,  # Lower temperature = more faithful transcription
            max_tokens=1024
        )
        return completion.choices[0].message.content

    except Exception as e:
        return f"Groq API Error: {str(e)}"

# --- 3. MAIN UI LAYOUT ---

st.title("📜 Historical Manuscript Digitizer")
st.markdown("**Pipeline:** Image → HTR through **Qwen2.5-VL** (Chinese/English) → and then NMT through an LLM ")

col_opt, col_file = st.columns([1, 2])
with col_opt:
    # UPDATED: Replaced Arabic with Chinese
    mode = st.radio("Manuscript Language", ["Chinese", "Old English"])

with col_file:
    uploaded_file = st.file_uploader("Upload Manuscript", type=['jpg', 'png', 'jpeg', 'tiff', 'tif'])

if uploaded_file and st.button("🚀 Digitize & Translate"):
        col1, col2 = st.columns(2)

        try:
            image = Image.open(uploaded_file).convert("RGB")
            with col1:
                # UPDATED: using use_container_width instead of use_column_width
                st.image(image, caption="Original Document", use_container_width=True)
        except Exception as e:
            st.error(f"Error opening image: {e}")
            st.stop()

        # --- PHASE 1: HTR (Local GPU) ---
        with st.spinner(f"Reading {mode} Handwriting (Qwen2.5-VL)..."):
            try:
                proc, model = load_qwen_model()
                raw_text = extract_text_qwen(image, proc, model, mode)
                st.toast("HTR Complete!", icon="✅")
            except Exception as e:
                st.error(f"HTR Error: {e}")
                st.stop()

        # --- PHASE 2: LLM (Cloud API) ---
        with st.spinner("Refining & Translating (LLM API)..."):
            try:
                result = process_with_llm(raw_text, mode)
                with col1:
                    st.subheader("📝 Extracted Text")
                    st.text_area("Raw Output", raw_text, height=300)
                with col2:
                    st.subheader("🌍 Translation")
                    st.markdown(result)
            except Exception as e:
                st.error(f"Translation Error: {e}")

Writing app.py


##  Launching Streamlit App + Cloudflare Tunnel (Public Demo Link)

This section:
1. Installs Cloudflared
2. Runs Streamlit on port **8501**
3. Creates a **temporary public URL** using Cloudflare Tunnel
4. Prints the live demo link (trycloudflare.com)


# **3. Launch App with Cloudflare Tunnel**

In [3]:
import time
import subprocess
import re
import sys

# 1. Install Cloudflared
print("Installing Cloudflared...")
!wget -q -O cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1

# 2. Run Streamlit in the background
print("Starting Streamlit App...")
# We redirect output to devnull so it doesn't clutter the console
# Updated startup command to disable CORS and XSRF protection
subprocess.Popen([
    "streamlit", "run", "app.py",
    "--server.port=8501",
    "--server.address=0.0.0.0",
    "--server.enableCORS=false",
    "--server.enableXsrfProtection=false",
    "--server.headless=true"
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

time.sleep(3)

# 3. Start the Cloudflare Tunnel
print("🚇 Creating Cloudflare Tunnel...")
# We start the tunnel and capture its output
process = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1,
    universal_newlines=True
)

# 4. Extract and Print the URL reliably
print("Scanning for URL...")
found_url = False
try:
    # Read stderr because cloudflared prints the URL there
    for line in iter(process.stderr.readline, ''):
        if ".trycloudflare.com" in line:
            # Use Regex to find the URL cleanly
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                print(f"\n🚀 \033[1m OUR APP IS LIVE HERE: {url} \033[0m")
                found_url = True
                break
except KeyboardInterrupt:
    print("Stopping...")
    process.terminate()

if not found_url:
    print("⚠️ Could not auto-detect URL. Please check the logs above.")

Installing Cloudflared...
Starting Streamlit App...
🚇 Creating Cloudflare Tunnel...
Scanning for URL...

🚀  OUR APP IS LIVE HERE: https://ver-mixing-revolution-satin.trycloudflare.com 


## ✅ Demo Checklist

During demo, show:
1. Upload a manuscript image
2. Select language mode (Chinese / Old English)
3. Click **Digitize & Translate**
4. Show:
   - Original image
   - Raw transcription output
   - Corrected + modern translation


